In [1]:
from py2neo import Graph
import pandas as pd

In [2]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [3]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [4]:
from py2neo import Node, Relationship

In [5]:
people = pd.read_csv('https://raw.githubusercontent.com/Lucy-Shichman/database-project/main/People.csv')
people

,Id,first_name,last_name,age,academic_id,employment_id,hobby_id,beef_id,restaurant_id
0,1,Lucy,Shichman,21,1,1,1,a,b
1,2,Himayat,Azizi,20,3,2,4,b,b
2,3,John,Le,21,2,4,5,d,c
3,4,Jay,Peetla,18,4,3,3,c,a
4,5,Franky,padilla coo,20,5,5,2,c,d


In [6]:
peopleNode = []
for i in range(len(people)):
    row = people.iloc[i]
    peopleNode.append(Node("Person",name=row['first_name'],lastName=row['last_name']))
    
peopleNode

[Node('Person', lastName='Shichman', name='Lucy'),
 Node('Person', lastName='Azizi', name='Himayat'),
 Node('Person', lastName='Le', name='John'),
 Node('Person', lastName='Peetla', name='Jay'),
 Node('Person', lastName='padilla coo', name='Franky')]

In [7]:
for i in range(len(peopleNode)):
    graph.create(peopleNode[i])

In [8]:
hobbiesCSV = pd.read_csv('https://raw.githubusercontent.com/Lucy-Shichman/database-project/main/Hobbies%20.csv')
hobbiesCSV

,hobby_id,hobby1,hobby2,organizations
0,1,Hiking,Rock Climbing,Clay Club
1,2,flag football,Tennis,OYFA
2,3,Basketball,Racing,ML Club
3,4,Basketball,Hiking,"DSAC, PCS, MSA"
4,5,Basketball,Photography,VSA


In [9]:
hobby1 = list(hobbiesCSV.hobby1.unique())
hobby2 = list(hobbiesCSV.hobby2.unique())

hobbies = list(set(hobby1 + hobby2))
hobbies

['Tennis',
 'Racing',
 'Photography',
 'Hiking',
 'flag football',
 'Basketball',
 'Rock Climbing']

In [10]:
hobbiesNode = {}
for i in range(len(hobbies)):
    hobbiesNode[hobbies[i]] = Node("Hobby",name=hobbies[i])
    
hobbiesNode

{'Tennis': Node('Hobby', name='Tennis'),
 'Racing': Node('Hobby', name='Racing'),
 'Photography': Node('Hobby', name='Photography'),
 'Hiking': Node('Hobby', name='Hiking'),
 'flag football': Node('Hobby', name='flag football'),
 'Basketball': Node('Hobby', name='Basketball'),
 'Rock Climbing': Node('Hobby', name='Rock Climbing')}

In [11]:
Does = Relationship.type('DOES')

In [12]:
for i in range(len(people)):
    idNum = people['hobby_id'][i]
    hobby1Temp = hobbiesCSV.loc[hobbiesCSV['hobby_id'] == idNum]['hobby1'].iloc[0]
    hobby2Temp = hobbiesCSV.loc[hobbiesCSV['hobby_id'] == idNum]['hobby2'].iloc[0]
    graph.create(Does(peopleNode[i],hobbiesNode[hobby1Temp]))
    graph.create(Does(peopleNode[i],hobbiesNode[hobby2Temp]))

In [13]:
employment = pd.read_csv('https://raw.githubusercontent.com/Lucy-Shichman/database-project/main/Employment.csv')
employment

,employment_id,company,city,role
0,1,FGH,DC,Analyst
1,2,Teaching Lab,NaN,Data Scientist
2,3,ENSCO,Richmond,Data Scientist
3,4,Premise,DC,Data Scientist
4,5,IT Concepts,Vienna,Data Scientist


In [14]:
employment.fillna('',inplace=True)
employmentNode = []
for i in range(len(employment)):
    row = employment.iloc[i]
    employmentNode.append(Node("Employment",company=row['company'],city=row['city'],role=row['role']))
    
employmentNode

[Node('Employment', city='DC', company='FGH', role='Analyst'),
 Node('Employment', city='', company='Teaching Lab', role='Data Scientist'),
 Node('Employment', city='Richmond', company='ENSCO', role='Data Scientist'),
 Node('Employment', city='DC', company='Premise', role='Data Scientist'),
 Node('Employment', city='Vienna', company='IT Concepts', role='Data Scientist')]

In [15]:
WorksFor = Relationship.type("WORKS_FOR")

In [16]:
for i in range(len(peopleNode)):
    graph.create(WorksFor(peopleNode[i],employmentNode[i]))

In [17]:
academics = pd.read_csv('https://raw.githubusercontent.com/Lucy-Shichman/database-project/main/Academics.csv')

major = academics.major.unique()
school = academics.school.unique()
minor_name = academics.school.unique()

academics

,academic_id,major,university,school,minor_existence,minor_name
0,1,Biology,UVA,CAS,True,ds
1,2,Econ,UVA,CAS,True,ds
2,3,CS,UVA,CAS,True,ds
3,4,CS,UVA,Engineering,False,NaN
4,5,CS,UVA,Engineering,True,ds


In [18]:
academics.fillna('', inplace=True)
academicsNode = {}
academicsNode2 = {}
for i in range(len(academics)):
    row = academics.iloc[i]
    academicsNode[row['major']] = Node("Major",major=row['major'])
    academicsNode2[row['school']] = Node("School",school=row['school'])
    
academicsNode

{'Biology': Node('Major', major='Biology'),
 'Econ': Node('Major', major='Econ'),
 'CS': Node('Major', major='CS')}

In [19]:
studies = Relationship.type("STUDIES")

In [20]:
for i in range(len(people)):
    idNum = people['academic_id'][i]
    majorTemp = academics.loc[academics['academic_id'] == idNum]['major'].iloc[0]
    schoolTemp = academics.loc[academics['academic_id'] == idNum]['school'].iloc[0]
    graph.create(studies(peopleNode[i],academicsNode[majorTemp]))
    graph.create(studies(peopleNode[i],academicsNode2[schoolTemp]))

In [21]:
restaurants = pd.read_csv('https://raw.githubusercontent.com/Lucy-Shichman/database-project/main/restaurant.csv')
restaurants

,restaurant_id,name,location,type
0,a,Chipotle,Barracks,Fast Food
1,b,Wendy's,Charlottesville,Fast Food
2,c,Taco Bell,Emmet St,Fast Food
3,d,Cook Out,Charlottesville,Fancy
4,e,Asados,Charlottesville,Fancy Shmancy


In [22]:
restaurants["name"][1] = 'Wendys'
restaurantsNode = {}
for i in range(len(restaurants)):
    row = restaurants.iloc[i]
    restaurantsNode[row['name']] = Node("restaurant",name=row['name'])

restaurantsNode

{'Chipotle': Node('restaurant', name='Chipotle'),
 'Wendys': Node('restaurant', name='Wendys'),
 'Taco Bell': Node('restaurant', name='Taco Bell'),
 'Cook Out': Node('restaurant', name='Cook Out'),
 'Asados': Node('restaurant', name='Asados')}

In [23]:
Hates = Relationship.type('HATES')

In [24]:
for i in range(len(people)):
    idNum = people['restaurant_id'][i]
    restaurantTemp = restaurants.loc[restaurants['restaurant_id'] == idNum]['name'].iloc[0]
    graph.create(Hates(peopleNode[i],restaurantsNode[restaurantTemp]))

In [25]:
beef = pd.read_csv('https://raw.githubusercontent.com/Lucy-Shichman/database-project/main/Beef.csv')
beef

,beef_id,beef_with,why_beef,friends_with,why_friends
0,a,Franky,drinks dasani,no one,watches stranger things
1,b,Lucy,doesn't mix chipotle bowl,Jay,likes chipotle
2,c,Jay,doesn't use turn signals,himayatullah,cool guy
3,d,John,loves the smell of stink,lucy,loves pineapple on pizza


In [26]:
peopleNodes = {}
for i in range(len(people)):
    row = people.iloc[i]
    peopleNodes[row['first_name']] = peopleNode[i]
    
peopleNodes

{'Lucy': Node('Person', lastName='Shichman', name='Lucy'),
 'Himayat': Node('Person', lastName='Azizi', name='Himayat'),
 'John': Node('Person', lastName='Le', name='John'),
 'Jay': Node('Person', lastName='Peetla', name='Jay'),
 'Franky': Node('Person', lastName='padilla coo', name='Franky')}

In [27]:
beef['friends_with'][0] = ''

beefNode = {}
for i in range(len(beef)):
    row = beef.iloc[i]
    beefNode[row['beef_id']] = peopleNodes[row['beef_with']]

beefNode

{'a': Node('Person', lastName='padilla coo', name='Franky'),
 'b': Node('Person', lastName='Shichman', name='Lucy'),
 'c': Node('Person', lastName='Peetla', name='Jay'),
 'd': Node('Person', lastName='Le', name='John')}

In [28]:
HasBeefWith = Relationship.type('HAS_BEEF_WITH')
FriendsWith = Relationship.type('IS_FRIENDS_WITH')

In [29]:
for i in range(len(people)):
    idNum = people['beef_id'][i]
    beefTemp = beef.loc[beef['beef_id'] == idNum]['beef_id'].iloc[0]
    #print(people['first_name'][i] + " " + str(beefNode[beefTemp]))
    graph.create(HasBeefWith(peopleNode[i],beefNode[beefTemp]))